In [ ]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import adfuller

In [ ]:
SP = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']) # This is S&P500
FTSE = pd.read_html('https://en.wikipedia.org/wiki/FTSE_100_Index')[4]['EPIC']
NAS = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')[4]['Ticker'] # this is NASDAQ

f_FTSE = []
for i in FTSE:
    new = i + '.L'
    f_FTSE.append(new)

symbols = list([*SP, *NAS])

In [ ]:
start = '2021-01-01'
data = yf.download(f_FTSE, start=start)['Adj Close']

In [ ]:
df = data.copy()
df = df.dropna(axis=1)

---
### ADF Test

In [ ]:
def adf_test(data):
    output = []
    for i in df:
        adf = adfuller(data[i])[1]
        if adf <= 0.05:
            output.append(i)
    return output

In [ ]:
adf = adf_test(df)

---
### Hurst Exponent

In [ ]:
import numpy as np

In [ ]:
def get_hurst_exponent(time_series, max_lag=20):
    """Returns the Hurst Exponent of the time series"""
    
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

In [ ]:
hurst_stocks = []

for i in df:
    temp_list = []
    for lag in [20, 100, 300, 500, 1000]:
        hurst_exp = get_hurst_exponent(df[i].values, lag)
        temp_list.append(hurst_exp)
        if all(j <= 0.45 for j in temp_list): # <0.5 is mean reverting
            hurst_stocks.append(i)
        else:
            continue
        
print(list(sorted(set(hurst_stocks))))

---
### Combined List

In [ ]:
fin = list(set(hurst_stocks) and set(adf))
print(fin)

The issue with these metrics and the book in general (Algorithmic Trading by Ernest P. Chang), is that there isn't much alpha in them. You aren't going to create a winning strategy using ADF and Hurst Exponent